# **Decoding Emotion in Music: Analyzing the Influence of Audio Features on Affective Perception Using Machine Learning**

## **1. Research Question and Motivation**
### **Can machine learning predict the emotional impact of a song based on its musical features?**

Music has a profound influence on emotions and mental well-being. Studies suggest that:
- Higher-pitched music (major keys, fast tempo) induces happiness and excitement.
- Lower-pitched music (minor keys, slow tempo) is associated with sadness or relaxation.
- Dissonant music may cause anxiety or stress.

Understanding the relationship between **pitch, key, tempo, and mood** can enhance **music therapy, mental health applications, and personalized music recommendations**. This study employs machine learning to analyze how audio features influence emotion perception in music.

### **Related Work**
- **Chanda & Levitin (2013):** Music influences **neurochemistry**, reducing stress (cortisol), boosting mood, and strengthening immunity.
- **Kraus & Chandrasekaran (2010):** **Music training** improves auditory processing, memory, and cognitive skills.
- **

## **2. Data Collection & Data Wrangling**

This section involves:
- Extracting music tags from the **Million Song Dataset**.
- Generating **Word2Vec embeddings** to classify music-related words into **emotion categories**.
- Connect to `tags.db` and extract matching tags’ last.fmID based on emotion mapping
- Extract matching tracks from `metadata.db` using last.fmID, getting tracks’ titles and artists’ names 
- Search SpotifyIDs by artists and titles and Store Spotify SpotifyIDs with labeled emotions



### Import libraries

In [4]:
import os
import sqlite3
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotify_secret import SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET
import pandas as pd
import gensim.downloader as api
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import nltk
from nltk.corpus import stopwords
import os
import time
from collections import deque

### Check Tags in `tags.db`

http://millionsongdataset.com/lastfm/#getting 

Full list of tags, ordered by frequency (which is also specified). The frequency is simply the number of tracks associated with each tag. We did not sum the count Last.fm provides (an integer between 0 and 100). The dataset includes tags related to music genres, moods, and styles. These tags will be embedded using Word2Vec for further analysis.

In [3]:
with sqlite3.connect('datasets/tags.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM tags LIMIT 20')
    data = cursor.fetchall()
    for row in data:
        print(row[0])
conn.close()

classic rock
Progressive rock
blues
memphis slim
pop
70s
Middle of the road
Bonjour ca va
Tony Levin
instrumental
Zappa Related
Bozzio Levin Stevens
Steve Stevens
groovy
cool
experimental
Experimental Rock
jazz fusion
hard progressive rock
Black Light Syndrome


### Word2Vec Optimization
* Extract all tags from tags.db and generate embeddings using the GoogleNews300 pretrained model.
* Compute the cosine similarity between each tag and the five core emotions: happy, sad, angry, fear, surprise.
* Rank tags based on similarity and select the top 20,000 most relevant tags for each emotion.
* Perform word frequency analysis on the selected tags and remove irrelevant stopwords (e.g., "lovely," "amazing," "awesome").

In [4]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Connect to tag database and get all tags
with sqlite3.connect('datasets/tags.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT tag FROM tags')
    all_tags = [row[0] for row in cursor.fetchall()]

print(f"Total number of tags in database: {len(all_tags)}")

# Download the GoogleNews word2vec model
print("Loading Word2Vec model...")
model = api.load("word2vec-google-news-300")
print("Model loaded successfully")

# Calcualte embeddings for all tags
tag_embeddings = {}
for tag in all_tags:
    words = tag.split()  # Split the tag into words
    word_vectors = [model[word] for word in words if word in model]  # Filter out OOV words
    if word_vectors:
        tag_embeddings[tag] = np.mean(word_vectors, axis=0)  # Use the mean of word vectors as tag embedding

print(f"Successfully generated embeddings for {len(tag_embeddings)} tags")
conn.close()

[nltk_data] Downloading package stopwords to /Users/vera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total number of tags in database: 522366
Loading Word2Vec model...
Model loaded successfully
Successfully generated embeddings for 405797 tags


In [5]:
# Calculate the similarity between tags and 5 emotion core words
emotion_categories = ["happy", "sad", "angry", "fear", "surprise"]
emotion_vectors = {emo: model[emo].reshape(1, -1) for emo in emotion_categories}

# Calculate the similarity score
similarities = {emo: {} for emo in emotion_categories}
for tag, embedding in tag_embeddings.items():
    for emo, emo_vector in emotion_vectors.items():
        similarity = cosine_similarity(emo_vector, embedding.reshape(1, -1))[0][0]
        similarities[emo][tag] = similarity

# Take out the top 50000 tags for each emotion category
top_n = 50000
top_tags_per_emotion = {}
for emo in emotion_categories:
    sorted_tags = sorted(similarities[emo], key=similarities[emo].get, reverse=True)[:top_n]
    top_tags_per_emotion[emo] = sorted_tags

In [6]:
# Calculate the most common words for each emotion category, and remove stopwords
filtered_top_tags = {}
for emo, tags in top_tags_per_emotion.items():
    words = [word for tag in tags for word in tag.split()]  # Split tag to single words
    filtered_words = [word for word in words if word not in stop_words]  # remove stopwords
    most_common_words = [word for word, _ in Counter(filtered_words).most_common(5000)]  # Pick 5000 high frequency words
    filtered_top_tags[emo] = most_common_words

# Generate raw emotion mapping
raw_emotion_mapping = {emo: tuple(filtered_top_tags[emo]) for emo in emotion_categories}

print("\**Generated Raw Emotion Mapping:**")
for emo, words in raw_emotion_mapping.items():
    print(f"{emo}: {words}")


\**Generated Raw Emotion Mapping:**
happy: ('love', 'I', 'good', 'song', 'like', 'songs', '-', 'great', 'want', 'music', 'nice', 'happy', 'awesome', 'get', 'makes', 'make', 'feel', 'sad', 'best', 'know', 'really', 'one', 'listen', 'dont', 'life', 'time', 'way', 'beautiful', 'go', 'never', 'bad', 'sweet', 'would', 'better', 'fun', 'amazing', 'live', 'wanna', 'fucking', 'ever', 'think', 'could', 'cool', 'back', 'dance', 'much', 'stuff', 'day', 'track', 'sing', 'little', 'got', 'still', 'pretty', 'feeling', 'come', 'say', 'crazy', 'cant', 'see', 'im', 'wish', 'play', 'shit', 'hard', 'oh', 'need', 'something', 'things', 'perfect', 'cry', 'reminds', 'yeah', 'heart', 'hear', 'loves', 'old', 'going', 'let', 'loved', 'lovely', 'right', 'band', 'always', 'u', 'new', 'long', 'hate', 'sounds', 'world', 'big', 'thing', 'favorite', 'fuck', 'baby', 'listening', 'dream', 'made', 'gonna', 'girl', 'smile', 'people', 'bit', 'heard', 'head', 'mood', 'sound', 'You', 'first', 'nothing', 'ok', 'upbeat', 'ho

<>:12: SyntaxWarning: invalid escape sequence '\*'
<>:12: SyntaxWarning: invalid escape sequence '\*'
/var/folders/jw/cc3wc0f13632_rh4jzxbdz3w0000gn/T/ipykernel_24336/2496750795.py:12: SyntaxWarning: invalid escape sequence '\*'
  print("\**Generated Raw Emotion Mapping:**")


# Note: ChatGPT Usage
I used ChatGPT for cleaning the raw emotion. My input is: Remove the words that is irrelevant to happy/sad/angry/fear/surprise and put them in emotional mapping format. 


### Cleaned Emotion Mapping to Categorize Words to 5 Labels - Happy, Sad, Agery, Fear, Surprise

Emotion Wheel by Idaho State University: https://www.isu.edu/media/libraries/counseling-and-testing/documents/Wheel-of-Emotions-Handout-(3).pdf

In [6]:
emotion_mapping = {
    "happy": (
        "love", "good", "great", "nice", "happy", "awesome", "beautiful", "fun", "amazing", "dance",
        "sweet", "better", "smile", "fantastic", "joy", "wonderful", "cheerful", "pleasure",
        "upbeat", "cool", "positive", "perfect", "uplifting", "exciting", "alive", "hopeful",
        "bright", "sunny", "laugh", "merry", "delight", "lucky", "sunshine", "enjoy", "bliss",
        "cheery", "satisfying", "warm", "friendly", "peaceful", "thrilled", "comforting",
        "relaxing", "soothing", "loved", "calm", "bubbly", "grateful", "lovely", "harmonious",
        "glorious", "cheer", "exhilarating", "carefree", "serene", "jolly", "radiant",
        "heartwarming", "charming", "affectionate", "giddy", "ecstatic", "gleeful", "rejoice",
        "euphoria", "miracle", "pleasant", "refreshing", "heavenly", "blessed", "celebrate",
        "adore", "cheers", "fascinating", "playful", "motivating", "appreciate", "energetic",
        "laughing", "wholesome", "enchanting", "lively", "optimistic", "thrill", "joyful",
        "excited", "electrifying", "joyful", "interested", "proud", "accepted", "powerful", 
        "peaceful", "intimate", "optimistic"
    ),
    "sad": (
        "sad", "sorrow", "heartbreak", "melancholy", "melancholic", "tragic", "pain", "painful",
        "heartache", "bittersweet", "mournful", "grief", "depressed", "depression", "lonely",
        "loneliness", "despair", "emptiness", "longing", "misery", "regrets", "hopeless", "hurting",
        "unbearable", "aching", "suffer", "desolate", "disappointment", "devastating",
        "tortured", "lament", "lamentation", "despairing", "dismal", "weep", "weeping", "tear",
        "tears", "tearful", "crying", "sob", "sobbing", "heartbroken", "loss", "shattered",
        "mourn", "mourning", "heartwrenching", "wistful", "nostalgia", "nostalgic", "bleak",
        "devastation", "solemn", "ache", "agony", "pity", "gloomy", "grief-stricken",
        "somber", "unsettling", "troubling", "poignant", "wretched", "despondent", "mournfully",
        "sorrowing", "dejected", "disheartened", "dispirited", "guilty", "abandoned", "despair",
        "depressed", "lonely", "bored"
    ),
    "angry": (
        "hate", "die", "bitch", "damn", "kill", "mad", "angry", "brutal", "rage", "killing", "riot",
        "revenge", "sucks", "scream", "brutality", "furious", "fucked", "fury", "chaotic", "violent",
        "attack", "wild", "aggressive", "harsh", "fight", "evil", "cruel", "sadistic", "hopeless",
        "suicidal", "shout", "dumb", "stupid", "sick", "drown", "bleeding", "nasty", "hatred",
        "insane", "venom", "hell", "bloody", "anarchy", "homicidal", "pissed", "burn", "sinister",
        "wretched", "hateful", "frenzy", "menacing", "crushing", "grudge", "screeching", "intense",
        "threatened", "hateful", "mad", "aggressive", "frustrated", "distant", "critical"
    ),
    "surprise" : (
        "unexpected", "unusual", "shock", "astonishing", "unexpectedly", "amazing", 
        "surprised", "stunned", "startling", "unexpectedly", "mindblowing", "unexpectedness",
        "unbelievable", "jaw-dropping", "spectacular", "incredible", "unforeseen", 
        "sudden", "remarkable", "miraculous", "outstanding", "eye-opening", "breathtaking",
        "thrilling", "wonder", "wow", "unanticipated", "curious", "unexpectedness",
        "surprise", "blow", "crazy", "fantastic", "wonderful", "magic", "mystery", 
        "strange", "unfamiliar", "spontaneous", "jumps", "wild", "shockingly",
        "astonished", "fluke", "random", "serendipity", "striking", "flabbergasted",
        "explosive", "spine-tingling", "extraordinary", "eye-popping", "triumph", 
        "bizarre", "peculiar", "unexplored", "fresh", "mystifying", "phenomenal",
        "wondrous", "outlandish", "peculiarity", "wonderstruck", "jawdrop", "awe",
        "unorthodox", "intriguing", "offbeat", "quirky", "mesmerizing", "unprecedented",
        "unconventional", "twist", "unusualness", "baffling", "unpredictable", 
        "adventure", "unaccustomed", "groundbreaking", "different", "enlightening",
        "curiosity", "discover", "elating", "suddenness", "mesmerize", "twisting",
        "unreal", "blowmind", "wowed", "unexpectedly", "overwhelming", "ecstatic",
        "amusement", "unknown", "newfound", "epiphany", "exploring", "realization"
    ),
    "fear": (
        "doom", "kill", "mad", "alone", "scream", "dead", "evil", "terror", "dark", "nightmare",
        "violence", "destroy", "slowly", "dying", "bleed", "ghost", "panic", "enemy",
        "misery", "heartache", "revenge", "despair", "insane", "tremble", "satan",
        "grief", "madness", "rage", "dread", "thrill", "night", "haunting", "afraid",
        "shock", "suffocate", "danger", "disturbing", "creepy", "brutal", "eerie",
        "frightened", "shadows", "worry", "guilt", "paranoia", "creatures", "chaos",
        "distress", "weird", "horrifying", "sinister", "anxiety", "tension", "freak",
        "gloom", "shiver", "unnerving", "nervous", "uneasy", "fearful", "menacing",
        "dreadful", "apprehension", "humiliated", "rejected", "submissive", "insecure", 
        "anxious", "scared"
    )
}

### Connect to `tags.db` and Extract the Matching Tags' ID based on the Emotional Mapping

Join `tags` table with `tids` table (`tids` contains track ids used in <b>Million Songs</b> database, and they are different from Spotify API's track ids

In [ ]:
tag_to_track_ids = {}
with sqlite3.connect("datasets/tags.db") as conn:
    cursor = conn.cursor()

    for tag, potential_names in emotion_mapping.items():
        sql = f"""
        SELECT tids.tid 
        FROM tid_tag
        JOIN tids ON tids.ROWID = tid_tag.tid
        JOIN tags ON tid_tag.tag = tags.ROWID
        WHERE ({' OR '.join(["tags.tag LIKE ?" for _ in potential_names])})
        """
        cursor.execute(sql, [f"%{word}%" for word in potential_names])
        data = cursor.fetchall()
        tag_to_track_ids[tag] = [row[0] for row in data]

print({k: len(v) for k, v in tag_to_track_ids.items()})

conn.close()

### Extract Matching Tracks

`track_metadata.db`: http://millionsongdataset.com/pages/getting-dataset/ 

MillionSongSubset/Additional Files/SQLite database containing most metadata about each track (NEW VERSION 03/27/2011).

Retrieve the corresponding title and artist name from the `songs` table (coming from <b>Million Songs</b> database) with the track ids obtained from previous step

In [18]:
with sqlite3.connect('datasets/track_metadata.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM songs LIMIT 10')
    data = cursor.fetchall()
    for row in data:
        print(row[0])
conn.close()

TRMMMYQ128F932D901
TRMMMKD128F425225D
TRMMMRX128F93187D9
TRMMMCH128F425532C
TRMMMWA128F426B589
TRMMMXN128F42936A5
TRMMMLR128F1494097
TRMMMBB12903CB7D21
TRMMMHY12903CB53F1
TRMMMML128F4280EE9


In [9]:
tag_to_track_info = {}

BATCH_SIZE = 500  # Process 500 track_ids at a time to avoid SQL variable limits

with sqlite3.connect("datasets/track_metadata.db") as conn:
    cursor = conn.cursor()
    
    for tag, tids in tag_to_track_ids.items():
        tag_to_track_info[tag] = set()  # Initialize empty set
        
        if tids:
            for i in range(0, len(tids), BATCH_SIZE):
                batch = tids[i:i + BATCH_SIZE]  # Take a batch of track_ids

                sql = f"""
                SELECT title, artist_name 
                FROM songs 
                WHERE track_id IN ({','.join(['?'] * len(batch))})
                """
                cursor.execute(sql, batch)
                data = cursor.fetchall()

                # Store results in the set (avoiding duplicates)
                tag_to_track_info[tag].update({(row[0], row[1]) for row in data})

print({k: len(v) for k, v in tag_to_track_info.items()})
print(f"Successfully retrieved track info for {len(tag_to_track_info)} emotion categories.")


{'happy': 178735, 'sad': 43625, 'angry': 109499, 'surprise': 60463, 'fear': 75010}
Successfully retrieved track info for 5 emotion categories.


### Set up Spotify API

https://spotipy.readthedocs.io/en/2.25.0/

`SPOTIFY_CLIENT_ID` and `SPOTIPY_CLIENT_SECRET` are under `spotify_secret.py` file.



In [10]:
os.environ['SPOTIPY_CLIENT_ID'] = SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = SPOTIPY_CLIENT_SECRET

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=30, retries=3)

### Search Spotify track ids by artists and titles and Store Spotify track ids to emotion in a dataframe and CSV file
The following block may take more than 30 mins to run since it will call the spotify API thousands of times to grab the corresponding Spotify track ids

- Check Existing Data: If track_id_to_emotion_data.csv exists, load the existing track IDs to avoid duplicate processing. Otherwise, start fresh.
- Iterate Through Each Emotion: Loop through each emotion category (e.g., happy, sad, angry, etc.). Select up to 10,000 tracks per emotion.
- Search for Track IDs on Spotify: Use Spotify’s API to find each track based on title + artist. If a track is found, extract its unique track ID.
- Avoid Duplicates: Check if the track ID already exists in the saved file. Only new track IDs are added.
- Save Data Incrementally: After each emotion is processed, append new results to track_id_to_emotion_data.csv. This avoids data loss if the script is interrupted.
- Prevent API Rate Limits: Add a small delay (0.2s per request) to prevent hitting Spotify's API limits. If an API error occurs, skip the track and continue.


In [13]:
import os
import time
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from requests.exceptions import ReadTimeout

# Define output file
OUTPUT_FILE = "track_id_to_emotion_data.csv"

# Define target counts for each emotion
target_counts = {
    "happy": 10000,
    "angry": 10000,
    "surprise": 10000,
    "fear": 10000,
    "sad": 10000,
}

# Load existing processed data
if os.path.exists(OUTPUT_FILE):
    track_id_to_emotion_pd = pd.read_csv(OUTPUT_FILE)
    processed_track_ids = set(track_id_to_emotion_pd["track_id"])
    
    # Count existing track IDs per emotion
    processed_emotion_counts = track_id_to_emotion_pd["emotion"].value_counts().to_dict()
else:
    processed_track_ids = set()
    processed_emotion_counts = {}

# Filter only emotions that still need more tracks
emotions_to_process = {
    emotion: count
    for emotion, count in target_counts.items()
    if processed_emotion_counts.get(emotion, 0) < count
}

if not emotions_to_process:
    print("All emotions have met their required count. No further processing needed.")
    exit()

print(f"Emotions still needing more data: {emotions_to_process}")

# Define API credentials (auto switch)
SPOTIFY_CREDENTIALS = [
    {"client_id": "SPOTIPY_CLIENT_ID_1", "client_secret": "SPOTIPY_CLIENT_SECRET_1"},
    {"client_id": "SPOTIPY_CLIENT_ID_2", "client_secret": "SPOTIPY_CLIENT_SECRET_2"},
]
current_cred_index = 0  # Start with first credential

def set_spotify_client(index):
    """ Switch to a new Spotify API client """
    global sp, current_cred_index
    creds = SPOTIFY_CREDENTIALS[index]
    os.environ["SPOTIPY_CLIENT_ID"] = creds["client_id"]
    os.environ["SPOTIPY_CLIENT_SECRET"] = creds["client_secret"]
    
    auth_manager = SpotifyClientCredentials()
    sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=5)
    print(f"Switched to API Credential {index + 1}")

# Initialize Spotify client
set_spotify_client(current_cred_index)

# Reverse mapping from tag to emotion
tag_to_emotion = {word: emotion for emotion, words in emotion_mapping.items() for word in words}

# Clean query function
def clean_query(title, artist):
    """ Ensure the query stays under 250 characters """
    max_query_length = 250
    artist = artist.split(";")[0].strip()  # Take only the first artist
    query = f"track:{title} artist:{artist}"
    
    if len(query) > max_query_length:
        title = title[:max_query_length - len(f"track: artist:{artist}") - 5] + "..."
        query = f"track:{title} artist:{artist}"
    
    return query

# Fetch track ID with auto-retry & quota handling
def fetch_track_id(title, artist, max_retries=3):
    """ Fetch track ID with retries and quota handling """
    global current_cred_index
    
    query = clean_query(title, artist)
    
    for attempt in range(max_retries):
        try:
            results = sp.search(q=query, type="track", limit=1)
            
            if results['tracks']['items']:
                return results['tracks']['items'][0]['id']
            else:
                return None  # No match found

        except spotipy.SpotifyException as e:
            error_msg = str(e).lower()
            
            # Handle rate limit / quota exhaustion
            if "rate limit" in error_msg or "quota" in error_msg:
                print("API Quota Limit Reached. Switching credentials...")

                # Switch API credentials automatically
                current_cred_index = (current_cred_index + 1) % len(SPOTIFY_CREDENTIALS)
                set_spotify_client(current_cred_index)

                time.sleep(5)  # Cooldown before retrying
            
            else:
                print(f"API Error: {e} - Skipping track")
                time.sleep(2)  # Small delay before retrying next track
                
        except ReadTimeout:
            print(f"Timeout Error - Retrying ({attempt + 1}/{max_retries})")
            time.sleep(5)  # Wait before retrying
    
    return None  # Return None if all retries fail

# Track already processed songs
seen_tracks = set()

# Process all emotions together (single query per track)
emotion_data = []
for emotion, tracks in tag_to_track_info.items():
    if emotion not in emotions_to_process:
        continue  # Skip if already full

    required_tracks = target_counts[emotion] - processed_emotion_counts.get(emotion, 0)
    print(f"Processing Emotion: {emotion} (Needs {required_tracks} more tracks)")

    for title, artist in tracks:
        if processed_emotion_counts.get(emotion, 0) >= target_counts[emotion]:
            print(f"{emotion} has reached the target of {target_counts[emotion]} tracks. Skipping further processing.")
            break

        # Skip if already processed
        if title in seen_tracks or title in processed_track_ids:
            continue

        # Find the most relevant emotion
        track_words = set(title.lower().split() + artist.lower().split())
        matched_emotions = [tag_to_emotion[word] for word in track_words if word in tag_to_emotion]

        if not matched_emotions:
            continue  # Skip if no match

        assigned_emotion = matched_emotions[0]  # Assign first matched emotion

        # Skip if that emotion is already full
        if assigned_emotion not in emotions_to_process:
            continue

        track_id = fetch_track_id(title, artist)

        if track_id:
            emotion_data.append((track_id, assigned_emotion))
            seen_tracks.add(title)
            processed_track_ids.add(track_id)
            processed_emotion_counts[assigned_emotion] = processed_emotion_counts.get(assigned_emotion, 0) + 1

        # Save after every 100 tracks
        if len(emotion_data) % 100 == 0 and emotion_data:
            df_new = pd.DataFrame(emotion_data, columns=["track_id", "emotion"])
            df_new.to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE), index=False)
            print(f"Saved {len(emotion_data)} new tracks")
            emotion_data = []

        time.sleep(0.5)

# Save remaining data
if emotion_data:
    df_new = pd.DataFrame(emotion_data, columns=["track_id", "emotion"])
    df_new.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
    print(f"Final save completed, {len(emotion_data)} new tracks added.")

print("Processing completed successfully.")


Emotions still needing more data: {'surprise': 10000}
Switched to API Credential 1
Processing Emotion: surprise (Needs 991 more tracks)
Saved 100 new tracks


Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
Saved 100 new tracks
surprise has reached the target of 10000 tracks. Skipping further processing.
Final save completed, 91 new tracks added.
Processing completed successfully.


### Generate Spotify track ids and audio features dataset from sqlite
https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features
https://onlyoneaman.medium.com/unleashing-the-power-of-audio-features-with-the-spotify-api-c544fda1af40


In [20]:
conn = sqlite3.connect("datasets/spotify.sqlite")
sql = f"""
    SELECT tracks.id AS track_id, features.acousticness,
    features.danceability, features.energy, features.instrumentalness, features.key,
    features.liveness, features.loudness, features.mode, features.speechiness, features.tempo, features.valence
    FROM audio_features features
    JOIN tracks ON features.id = tracks.audio_feature_id
    """
df = pd.read_sql_query(sql, conn)
conn.close()

In [21]:
print(df.head())
df.to_csv("spotify_tracks_with_features.csv", index=False)

                 track_id  acousticness  danceability  energy  \
0  2jKoVlU7VAmExKJ1Jh3w9P        0.1800         0.893   0.514   
1  4JYUDRtPZuVNi7FAnbHyux        0.2720         0.520   0.847   
2  6YjKAkDYmlasMqYw73iB0w        0.0783         0.918   0.586   
3  2YlvHjDb4Tyxl4A1IcDhAe        0.5840         0.877   0.681   
4  3UOuBNEin5peSRqdzvlnWM        0.1700         0.814   0.781   

   instrumentalness  key  liveness  loudness  mode  speechiness       tempo  \
0          0.000000   11    0.0596    -5.080     1        0.283   95.848000   
1          0.000000    9    0.3250    -5.300     1        0.427  177.371002   
2          0.000000    1    0.1450    -2.890     1        0.133   95.516998   
3          0.000000    1    0.1190    -6.277     0        0.259   94.834999   
4          0.000518   11    0.0520    -3.330     1        0.233   93.445000   

   valence  
0    0.787  
1    0.799  
2    0.779  
3    0.839  
4    0.536  


In [31]:
track_id_to_audio_features_pd = pd.read_csv('datasets/spotify_tracks_with_features.csv')
track_id_to_audio_features_pd.head()

,track_id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2jKoVlU7VAmExKJ1Jh3w9P,0.1800,0.893,0.514,0.000000,11,0.0596,-5.080,1,0.283,95.848000,4,0.787
1,4JYUDRtPZuVNi7FAnbHyux,0.2720,0.520,0.847,0.000000,9,0.3250,-5.300,1,0.427,177.371002,4,0.799
2,6YjKAkDYmlasMqYw73iB0w,0.0783,0.918,0.586,0.000000,1,0.1450,-2.890,1,0.133,95.516998,4,0.779
3,2YlvHjDb4Tyxl4A1IcDhAe,0.5840,0.877,0.681,0.000000,1,0.1190,-6.277,0,0.259,94.834999,4,0.839
4,3UOuBNEin5peSRqdzvlnWM,0.1700,0.814,0.781,0.000518,11,0.0520,-3.330,1,0.233,93.445000,4,0.536


### Join two tables by track ids to link emotions to audio features

In [32]:
merged_df = pd.merge(track_id_to_emotion_pd, track_id_to_audio_features_pd, on="track_id", how="inner")
merged_df.to_csv("datasets/merged_emotion_audio_features.csv", index=False)
merged_df.head()

,track_id,emotion,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,3J0tOqhh6tQJJty0diKeW2,happy,0.1390,0.489,0.762,0.000405,0,0.0819,-3.947,1,0.0321,134.503998,4,0.634
1,2uPgHqTZotNvcMVWuwkf5s,happy,0.0682,0.425,0.779,0.000299,9,0.0679,-7.757,1,0.1150,146.300003,4,0.594
2,6XjsKdQbCVCtOjioJztgGA,happy,0.0721,0.319,0.835,0.000287,0,0.0490,-3.422,1,0.0440,169.835007,4,0.335
3,4wQHk151amFYAuR6LNbBK7,happy,0.1520,0.442,0.932,0.000681,4,0.3380,-3.358,1,0.0763,130.723007,4,0.844
4,1E5ZAgjqiDdnzfe4E52ghX,happy,0.4750,0.736,0.891,0.002340,7,0.0741,-13.207,1,0.0823,107.936996,4,0.940


In [33]:
merged_df['emotion'].value_counts()

emotion
sad         5234
angry       4978
happy       4831
fear        4759
surprise    4609
Name: count, dtype: int64